In [1]:
from sklearn import tree
import pandas as pd
import os

In [2]:
activities_csv = pd.read_csv('ac_activities_torben_clean.csv')
activities_csv.head()

,id,type,start_date_local,x_start_h,name,x_min,x_mi,x_min/mi,x_mph,x_max_mph,...,athlete_count,average_temp,distance,elapsed_time,start_latitude,start_longitude,timezone,x_date,x_elev_%,x_url
0,868156484,Run,15.01.2017 06:17:41,6.3,Morning Run,248.2,26.510,9.36,6.41,9.17,...,29,NaN,42663.3,14892,33.176906,-117.335854,(GMT-08:00) America/Los_Angeles,2017-01-15,0.51,https://www.strava.com/activities/868156484
1,868156415,Run,17.01.2017 18:50:18,18.8,Evening Run,16.1,2.023,7.95,7.55,11.41,...,30,NaN,3256.2,1300,32.720301,-117.150205,(GMT-08:00) America/Los_Angeles,2017-01-17,NaN,https://www.strava.com/activities/868156415
2,868156459,Run,19.01.2017 06:35:12,6.6,Morning Run,87.0,10.091,8.62,6.96,9.62,...,1,NaN,16240.6,5271,32.853794,-116.960111,(GMT-08:00) America/Los_Angeles,2017-01-19,0.85,https://www.strava.com/activities/868156459
3,868156435,Run,21.01.2017 07:17:12,7.3,Morning Run,88.8,11.518,7.71,7.78,9.17,...,2,NaN,18535.6,5804,32.758368,-117.242017,(GMT-08:00) America/Los_Angeles,2017-01-21,0.16,https://www.strava.com/activities/868156435
4,868156458,Run,22.01.2017 16:50:51,16.8,Afternoon Run,89.6,10.066,8.90,6.74,8.50,...,1,NaN,16199.8,5658,32.853570,-116.960471,(GMT-08:00) America/Los_Angeles,2017-01-22,0.48,https://www.strava.com/activities/868156458


In [3]:
activities = activities_csv[["x_start_h", "name", "x_min", "x_mi", "x_min/mi", "total_elevation_gain", "average_cadence", "elev_low", "elev_high", "athlete_count", "distance", "elapsed_time"]].copy()
activities.head()

,x_start_h,name,x_min,x_mi,x_min/mi,total_elevation_gain,average_cadence,elev_low,elev_high,athlete_count,distance,elapsed_time
0,6.3,Morning Run,248.2,26.510,9.36,217.6,80.8,-7.1,97.1,29,42663.3,14892
1,18.8,Evening Run,16.1,2.023,7.95,0.0,84.5,40.7,44.6,30,3256.2,1300
2,6.6,Morning Run,87.0,10.091,8.62,137.6,84.0,106.0,201.5,1,16240.6,5271
3,7.3,Morning Run,88.8,11.518,7.71,29.5,84.9,1.3,20.1,2,18535.6,5804
4,16.8,Afternoon Run,89.6,10.066,8.90,77.0,83.0,96.0,137.9,1,16199.8,5658


In [4]:
activities = activities.dropna(how="any")
#activities["start_date_local"] = activities["start_date_local"].apply(pd.to_datetime,errors="coerce")
activities.dtypes

x_start_h               float64
name                     object
x_min                   float64
x_mi                    float64
x_min/mi                float64
total_elevation_gain    float64
average_cadence         float64
elev_low                float64
elev_high               float64
athlete_count             int64
distance                float64
elapsed_time              int64
dtype: object

In [5]:
activities.head()

,x_start_h,name,x_min,x_mi,x_min/mi,total_elevation_gain,average_cadence,elev_low,elev_high,athlete_count,distance,elapsed_time
0,6.3,Morning Run,248.2,26.510,9.36,217.6,80.8,-7.1,97.1,29,42663.3,14892
1,18.8,Evening Run,16.1,2.023,7.95,0.0,84.5,40.7,44.6,30,3256.2,1300
2,6.6,Morning Run,87.0,10.091,8.62,137.6,84.0,106.0,201.5,1,16240.6,5271
3,7.3,Morning Run,88.8,11.518,7.71,29.5,84.9,1.3,20.1,2,18535.6,5804
4,16.8,Afternoon Run,89.6,10.066,8.90,77.0,83.0,96.0,137.9,1,16199.8,5658


In [6]:
target = activities["name"]
target_names = ["Morning Run","Lunch Run", "Afternoon Run","Evening Run","Night Run","Morning Activity","Lunch Activity", "Afternoon Activity","Evening Activity","Night Activity"]

In [7]:
data = activities.drop("name", axis=1)
feature_names = data.columns
data.head()

,x_start_h,x_min,x_mi,x_min/mi,total_elevation_gain,average_cadence,elev_low,elev_high,athlete_count,distance,elapsed_time
0,6.3,248.2,26.510,9.36,217.6,80.8,-7.1,97.1,29,42663.3,14892
1,18.8,16.1,2.023,7.95,0.0,84.5,40.7,44.6,30,3256.2,1300
2,6.6,87.0,10.091,8.62,137.6,84.0,106.0,201.5,1,16240.6,5271
3,7.3,88.8,11.518,7.71,29.5,84.9,1.3,20.1,2,18535.6,5804
4,16.8,89.6,10.066,8.90,77.0,83.0,96.0,137.9,1,16199.8,5658


In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target, random_state=42)

In [9]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.9908256880733946

In [10]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=200)
rf = rf.fit(X_train, y_train)
rf.score(X_test, y_test)

0.9724770642201835

In [11]:
sorted(zip(rf.feature_importances_, feature_names), reverse=True)

[(0.5632279530307536, 'x_start_h'),
 (0.06296610449555319, 'elapsed_time'),
 (0.055981152384121984, 'x_mi'),
 (0.053005881985516846, 'x_min'),
 (0.052134450747468114, 'average_cadence'),
 (0.04996115631011963, 'elev_high'),
 (0.049577260070501505, 'distance'),
 (0.04478869744206548, 'elev_low'),
 (0.03168132459906818, 'x_min/mi'),
 (0.03132142752437345, 'total_elevation_gain'),
 (0.005354591410458115, 'athlete_count')]

In [12]:
data = activities.drop(["name", "x_start_h"], axis=1)
feature_names = data.columns
data.head()

,x_min,x_mi,x_min/mi,total_elevation_gain,average_cadence,elev_low,elev_high,athlete_count,distance,elapsed_time
0,248.2,26.510,9.36,217.6,80.8,-7.1,97.1,29,42663.3,14892
1,16.1,2.023,7.95,0.0,84.5,40.7,44.6,30,3256.2,1300
2,87.0,10.091,8.62,137.6,84.0,106.0,201.5,1,16240.6,5271
3,88.8,11.518,7.71,29.5,84.9,1.3,20.1,2,18535.6,5804
4,89.6,10.066,8.90,77.0,83.0,96.0,137.9,1,16199.8,5658


In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target, random_state=42)

In [14]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.6422018348623854

In [15]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=200)
rf = rf.fit(X_train, y_train)
rf.score(X_test, y_test)

0.6238532110091743

In [16]:
sorted(zip(rf.feature_importances_, feature_names), reverse=True)

[(0.13458770822218702, 'elapsed_time'),
 (0.12491635908365616, 'elev_high'),
 (0.12258028710468138, 'average_cadence'),
 (0.11952869675157583, 'elev_low'),
 (0.11381582950535606, 'x_min'),
 (0.10092432457584788, 'distance'),
 (0.0926269923994797, 'x_mi'),
 (0.08901467253463657, 'x_min/mi'),
 (0.08597544546119786, 'total_elevation_gain'),
 (0.01602968436138147, 'athlete_count')]